TOUTES LES COMPLICATIONS

In [2]:
import pandas as pd
import numpy as np 

# pour gérer les path sous windows

import pathlib
import os 
import shutil
from os import listdir
from os.path import isfile, join
import glob

# Pour le modèle 

import matplotlib
import matplotlib.pyplot as plt
import os
from time import time
import numpy as np
import pylab as pl
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import itertools
import shutil
from sklearn import preprocessing
from  sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# ETAPE 1 = PREPOCESSING : à ne faire qu'une fois, si les dossiers ont déjà été créés et qu'on a déjà commencé l'apprentissage il faut passer directement à l'étape 2.

# Créer les dossiers avec l'architecture requise

In [3]:
# train, dev et unlabeled

path = "train/clac"
os.makedirs(path)

path = os.getcwd()

path = "train/pas_clac"
os.makedirs(path)

path = "dev/clac"
os.makedirs(path)

path = "dev/pas_clac"
os.makedirs(path)

path = "unlabeled/unlabeled"
os.makedirs(path)


# all_clac et all_pas_clac pour le shuffle

path_all_clac = "./all_clac"
os.makedirs(path_all_clac)

path_all_pas_clac = "./all_pas_clac"
os.makedirs(path_all_pas_clac)

In [ ]:
Il faut mettre quelques exemples de clac et de pas clac annotés à la main dans les fichiers dev et train 
avant de commencer. On peut utiliser les exemples rangés dans les dossiers exemple_de_clac et exemple_de_pas_clac.


In [4]:
# 1) CHARGER EXTRACTION TOUTES LES COMPLICATIONS
# Il faut que la première ligne soit IPP DDK COMPLICATION1 COMPLICATIONX

extraction_tc = pd.read_excel('extraction_tc.xlsx')

df_extraction_tc = pd.DataFrame(extraction_tc)

df_stacked1 = df_extraction_tc.set_index(['IPP', 'DDK']).stack()

extraction_tc2 = pd.read_excel('extraction_tc2.xlsx')

df_extraction_tc2 = pd.DataFrame(extraction_tc2)

df_stacked2 = df_extraction_tc2.set_index(['IPP', 'DDK']).stack()

extraction_tc3 = pd.read_excel('extraction_tc3.xlsx')

df_extraction_tc3 = pd.DataFrame(extraction_tc3)

df_stacked3 = df_extraction_tc3.set_index(['IPP', 'DDK']).stack()

df_stacked = pd.concat([df_stacked1, df_stacked2, df_stacked3])

In [5]:
# 2) Pour créer une TABLE DE CORRESPONDANCE

df_stacked = df_stacked.reset_index()

df_stacked.rename(columns={0:'texte_complication', 'level_2':'type_complication'}, inplace=True)

df_stacked.to_excel('table_de_correspondance.xlsx')

In [6]:
# 3) PREPROCESSING du texte maintenant pour qu'il s'applique à tous les exemples
# forcer texte_complication à être du str et créer df_stacked.xlsx
df_stacked = df_stacked.astype({'texte_complication': str}).dropna(subset=['texte_complication'])
df_stacked.to_excel('df_stacked.xlsx')

# Pour enlever la ponctuaction et les mots vides de sens
# note pour plus tard : enlever les mots rares

noise_list = [",", ".", "?", ";", ":", "/", "!", "-", "+", "ÿ", "à"] 
word_noise_list = ["a", "ainsi", "assez", "au", "aux", "ce", "ceci", "cela",
                   "car", "ces", "cette", "ce", "celle", "du", "en", "il", "ils"
                   "elle", "elles", "que", "qui", "qu'", "se", "son", "sa", "ses",
                   "le", "la", "les", "l'", "un", "une", "de", "des", "au", "du"]


def remove_noise(input_text):
    noise_free_words = [caracter for caracter in input_text.lower() if caracter not in noise_list]
    noise_free_text = "".join(noise_free_words) 
    word_list = [word for word in noise_free_text.split(" ") if word not in word_noise_list]
    ouput_text = " ".join(word_list)
    return ouput_text

df_stacked['texte_complication'] = df_stacked['texte_complication'].apply(remove_noise)

In [8]:
# 4) SPLIT EXPORT EN DEV ET MODEL SETS
# Mettre 10% des exemples au hasard de coté, il faudra les labeliser à la main après
# Pour avoir à la fin un test set representatif de la population 
# (Et pas les 1000 derniers KT pour le test set)

# Pour savoir combien d'exemples ils faut mettre de côté
df_stacked
index = df_stacked.index
number_of_rows = len(index)
x = number_of_rows // 10
print("Il faut mettre de coté :", x, "exemples.")

# Pour mettre de côté, dans un xlsx séparé, ces x exemples
df_test = df_stacked.sample(n=x, random_state=1)
df_test.to_excel("test_unlabeled.xlsx") 

# Les colonnes qui ne sont pas communes aux deux df (donc l'extraction - le test) vont servir pour le model
df_model = pd.concat([df_stacked, df_test]).drop_duplicates(keep=False)

#### Pour résumer on a crée :

# un fichier excel avec 10% des exemples qu'il faudra fusionner avec les autres extractions 
# + labeliser à la main pour faire un test  set. 
# avec les autres exemples : une df "df_model" à partir de laquelle il faut sortir un dev set et un training set
# on va les labeliser là

Il faut mettre de coté : 1771 exemples.


In [ ]:
# Pour enlever les exemples qui sont déjà dans le dev, le test et le train set

test_id_list = list(pd.read_excel("test_unlabeled.xlsx", index_col=0).index)

labeled_files = []
for directory in ['train/clac', 'train/pas_clac', 'dev/clac', 'dev/pas_clac' ]:
    labeled_files += [str(f[:-4]) for f in listdir(directory) if f != 'desktop.ini' if isfile(join(directory, f))]
print(labeled_files)

In [13]:
# enlever de df_stacked, les exemples dans test ou labelises 

all_indices_to_removes = test_id_list + labeled_files

indices_to_keep = df_stacked.index.difference(all_indices_to_removes)
df_stacked = df_stacked.loc[indices_to_keep]
df_stacked.to_excel('df_stacked.xlsx')

In [14]:
# 5) MODEL -> DOSSIER UNLABELED
# création, dans le dossier unlabeled de fichier csv contenant l'exemple et nommé d'après l'index de la 
# table de correspondance

directory = pathlib.Path().absolute()
DATA_FOLDER = directory
TRAIN_FOLDER = os.path.join(DATA_FOLDER, "train")
DEV_FOLDER = os.path.join(DATA_FOLDER, "dev")
UNLABELED_FOLDER = os.path.join(DATA_FOLDER, "unlabeled")
ENCODING = 'utf-8'
categories = ['clac', 'pas_clac']

for idx, row_data in df_stacked.iterrows():
    file_name = os.path.join(UNLABELED_FOLDER, 'unlabeled')
    file_name = os.path.join(os.path.join(file_name), str(idx) + '.txt')
       
    create_file = open(file_name, "w+", encoding='utf-8')
    create_file.write(row_data['texte_complication'])
    create_file.close()

## ETAPE 2 : Entraîner le modèle

In [ ]:
# Version SANS balance des classes

directory = pathlib.Path().absolute()

NUM_QUESTIONS = 1 #nombre d'exemple qu'il propose à chaque tour
PLOT_RESULTS = True
ACTIVE = True
DATA_FOLDER = directory
TRAIN_FOLDER = os.path.join(DATA_FOLDER, "train") 
DEV_FOLDER = os.path.join(DATA_FOLDER, "dev") 
UNLABELED_FOLDER = os.path.join(DATA_FOLDER, "unlabeled")
ENCODING = 'utf-8'
while True:
    data_train = load_files(TRAIN_FOLDER, encoding=ENCODING)
    data_dev = load_files(DEV_FOLDER, encoding=ENCODING)
    data_unlabeled = load_files(UNLABELED_FOLDER, encoding=ENCODING)
    categories = data_train.target_names
    
    def size_mb(docs):
        return sum(len(s.encode('utf-8')) for s in docs) / 1e6
    
    data_train_size_mb = size_mb(data_train.data)
    data_dev_size_mb = size_mb(data_dev.data)
    data_unlabeled_size_mb = size_mb(data_unlabeled.data)
    
    print("%d documents - %0.3fMB (training set)" % (
        len(data_train.data), data_train_size_mb))
    print("%d documents - %0.3fMB (dev set)" % (
        len(data_dev.data), data_dev_size_mb))
    print("%d documents - %0.3fMB (unlabeled set)" % (
        len(data_unlabeled.data), data_unlabeled_size_mb))
    print("%d categories" % len(categories))
    print()
    y_train = data_train.target
    y_dev =  data_dev.target
    
    
    print("Extracting features from the training dataset using a sparse vectorizer")
    t0 = time()
    vectorizer = TfidfVectorizer(encoding= ENCODING, use_idf=True, norm='l1', binary=False, sublinear_tf=True, min_df=0.001, max_df=1.0, ngram_range=(1, 2), analyzer='word', stop_words=None)
    
    # the output of the fit_transform (x_train) is a sparse csc matrix.
    X_train = vectorizer.fit_transform(data_train.data)
    duration = time() - t0
    # print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_train.shape)
    print()
    
    print("Extracting features from the dev dataset using the same vectorizer")
    t0 = time()
    X_dev = vectorizer.transform(data_dev.data)
    duration = time() - t0
    # print("done in %fs at %0.3fMB/s" % (duration, data_dev_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_dev.shape)
    print()
    
    print("Extracting features from the unlabled dataset using the same vectorizer")
    t0 = time()
    X_unlabeled = vectorizer.transform(data_unlabeled.data)
    duration = time() - t0
    # print("done in %fs at %0.3fMB/s" % (duration, data_unlabeled_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_unlabeled.shape)
    print()
    

    
    
    def trim(s):
        """Trim string to fit on terminal (assuming 80-column display)"""
        return s if len(s) <= 80 else s[:77] + "..."
    
    ###############################################################################
    # Benchmark classifiers
    def benchmark(clf, X_train, X_dev, y_train, y_dev, X_unlabeled):
        print('_' * 80)
        print("Training: ")
        print(clf)
        t0 = time()
        
        # Create a scaler fitted to X_train to later standarize all the subsets with the same scale ------------------
        scaler = preprocessing.StandardScaler(with_mean=False)
        scaler = scaler.fit(X_train)

        X_train = scaler.transform(X_train)  # Standardizing ------------------     
        clf.fit(X_train, y_train)
        
        train_time = time() - t0
        print("train time: %0.3fs" % train_time)
    
        t0 = time()
        X_dev = scaler.transform(X_dev) # Standardizing ------------------
        pred = clf.predict(X_dev) 
        dev_time = time() - t0
        print("dev time:  %0.3fs" % dev_time)
    
        score = metrics.f1_score(y_dev, pred)
        accscore = metrics.accuracy_score(y_dev, pred)
        print ("pred count is %d" %len(pred))
        print ('accuracy score:     %0.3f' % accscore)
        print("f1-score:   %0.3f" % score)
    
        if hasattr(clf, 'coef_'):
            print("dimensionality: %d" % clf.coef_.shape[1])
            print("density: %f" % density(clf.coef_))
    
            
        
        #print("classification report:")
        #print(metrics.classification_report(y_dev, pred,
                                                #target_names=categories))
    
        
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_dev, pred))
        tn, fp, fn, tp = confusion_matrix(y_dev, pred).ravel()
        print("tn :", tn, "fp :", fp, "fn :", fn, "tp :", tp)
        
        # Plot non-normalized confusion matrix
        titles_options = [("Confusion matrix, without normalization", None),
                          ("Normalized confusion matrix", 'true')]
        for title, normalize in titles_options:
            disp = plot_confusion_matrix(clf, X_dev, y_dev,
                                 display_labels=categories,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
            disp.ax_.set_title(title)

            print(title)
            print(disp.confusion_matrix)

        plt.show()
        
        print("confidence for unlabeled data:")
        
        X_unlabeled = scaler.transform(X_unlabeled) # Standardizing ------------------
        
        #compute absolute confidence for each unlabeled sample in each class
        confidences = np.abs(clf.decision_function(X_unlabeled))
        #average abs(confidence) over all classes for each unlabeled sample (if there is more than 2 classes)
        if(len(categories) > 2):
            confidences = np.average(confidences, axix=1)
        
        print(confidences)
        sorted_confidences = np.argsort(confidences)
        question_samples = []
        #select top k low confidence unlabeled samples
        low_confidence_samples = sorted_confidences[0:NUM_QUESTIONS]
        #select top k high confidence unlabeled samples
        high_confidence_samples = sorted_confidences[-NUM_QUESTIONS:]

        question_samples.extend(low_confidence_samples.tolist())
        question_samples.extend(high_confidence_samples.tolist())

        
        print()
        clf_descr = str(clf).split('(')[0]
        return clf_descr, score, train_time, dev_time, question_samples
    
    
    results = []
    results.append(benchmark(LinearSVC(loss='l2', penalty='l2', 
                                                dual=False, tol=1e-3, class_weight='balanced'), 
                                                 X_train, X_dev, y_train, y_dev, X_unlabeled))
 
    
    # make some plots
    
    indices = np.arange(len(results))
    
    results = [[x[i] for x in results] for i in range(5)]
    
    clf_names, score, training_time, dev_time, question_samples = results
    training_time = np.array(training_time) / np.max(training_time)
    dev_time = np.array(dev_time) / np.max(dev_time)
    
    #if PLOT_RESULTS:
        #pl.figure(figsize=(12,8))
        #pl.title("Score")
        #pl.barh(indices, score, .2, label="score", color='r')
        #pl.barh(indices + .3, training_time, .2, label="training time", color='g')
        #pl.barh(indices + .6, dev_time, .2, label="dev time", color='b')
        #pl.yticks(())
        #pl.legend(loc='best')
        #pl.subplots_adjust(left=.25)
        #pl.subplots_adjust(top=.95)
        #pl.subplots_adjust(bottom=.05)
        
        #for i, c in zip(indices, clf_names):
        #    pl.text(-.3, i, c)
        #pl.savefig('ngramoptimize.png')
        #pl.show()

    if ACTIVE:
        for i in question_samples[0]:
            filename = data_unlabeled.filenames[i]
            print (filename)
            print ('**************************content***************************')
            print (data_unlabeled.data[i])
            print ('**************************content end***********************')
            print ("Annotate this text (select one label):")
            for i in range(0, len(categories)):
                print ("%d = %s" %(i+1, categories[i]))
            labelNumber = input("Enter the correct label number:")
            while labelNumber.isdigit()== False:
                labelNumber = input("Enter the correct label number (a number please):")
            labelNumber = int(labelNumber)
            category = categories[labelNumber - 1] 
            dstDir = os.path.join(TRAIN_FOLDER, category) 
            shutil.move(filename, dstDir)
            
        # shuffle pour mélanger les exemples dans dev et train à chaque cycle
        # sinon on va avoir un modèle sur psécialiser dans la classification des clac initiales 
            
            dstDir = os.path.join("./all_clac") 
            srcDir = os.path.join("./train/clac/")
            # selectionner les fichiers texte à déplacer
            files = [f for f in glob.glob(srcDir + "/*.txt", recursive=True)]

            #for f in files:
             #   print(f)

            # déplacer les fichier txt
            for f in files:
                shutil.move(f, dstDir)
                
            # pour le dev
            # définir le dossier source et le dossier destination
            dstDir = os.path.join("./all_clac") 
            srcDir2 = os.path.join("./dev/clac")

            # selectionner les fichiers texte à déplacer
            files = [f for f in glob.glob(srcDir2 + "/*.txt", recursive=True)]

            #for f in files:
             #   print(f)

            # déplacer les fichier txt
            for f in files:
                shutil.move(f, dstDir)
            
            #2) mettre tous les fichiers pas_clac de dev et de train dans le même dossier all_pas_clac 
            

            # déplacer tous les fichier.txt des dossiers train/pas_clac et dev/pas_clac dans all_pas_clac

            # pour le train
            # définir le dossier source et le dossier destination
            dstDir = os.path.join("./all_pas_clac") 
            srcDir = os.path.join("./train/pas_clac/")

            # selectionner les fichiers texte à déplacer
            files = [f for f in glob.glob(srcDir + "/*.txt", recursive=True)]

            #for f in files:
            #    print(f)

            # déplacer les fichier txt
            for f in files:
                shutil.move(f, dstDir)
    
            # pour le dev
            # définir le dossier source et le dossier destination
            dstDir = os.path.join("./all_pas_clac") 
            srcDir2 = os.path.join("./dev/pas_clac")

            # selectionner les fichiers texte à déplacer
            files = [f for f in glob.glob(srcDir2 + "/*.txt", recursive=True)]

            #for f in files:
           #     print(f)

            # déplacer les fichier txt
            for f in files:
                shutil.move(f, dstDir)
            
            srcDir_all_clac = os.path.join("./all_clac")
            srcDir_all_pas_clac = os.path.join("./all_pas_clac")

            # Nombre d'exemples qui sont bien des complications du cathéterisme
           
            no_clac = len(os.listdir(srcDir_all_clac))

            print("Nombre de complications liées au cathéterisme :")
            print(no_clac)

             # Nombre d'exemples qui ne sont pas des complications du cathéterisme

            no_pas_clac = len(os.listdir(srcDir_all_pas_clac))

            print("Nombre d'exemples qui ne sont pas des complications liées au cathéterisme :")
            print(no_pas_clac)


            # Nombre total d'exemples 

            no_total = len( (os.listdir(srcDir_all_clac)) + (os.listdir(srcDir_all_pas_clac)) )

            print("Total :")  
            print(no_total)
            
            s_dev_clac = int(no_clac * 0.3)
            
            print("Nombre d'exemples de clac pour le dev set :")
            print(s_dev_clac)

            s_dev_pas_clac = int(no_pas_clac * 0.3)

            print("Nombre d'exemples de pas clac pour le dev set :")
            print(s_dev_pas_clac)

            s_train_clac = no_clac - s_dev_clac

            print("Nombre d'exemples de clac pour le train set :")
            print(s_train_clac)

            s_train_pas_clac = no_pas_clac - s_dev_pas_clac

            print("Nombre d'exemples de pas clac pour le train set :")
            print(s_train_pas_clac)
            
            s_train = s_train_clac + s_train_pas_clac

            print("Nombre d'exemples pour le train set :")
            print(s_train)

            s_dev = s_dev_clac + s_dev_pas_clac

            print("Nombre d'exemples pour le dev set :")
            print(s_dev)

        
            def move(srcDir, dstDir, share=None):
                files = [f for f in glob.glob(srcDir + "/*.txt", recursive=True)]
                if share is None:
                    share =  len(files)
                for f in random.sample(files, share):
                    shutil.move(f, dstDir)
                    
            # Pour le dossier dev_clac

            srcDir = os.path.join("./all_clac")
            dstDir = os.path.join("./dev/clac")
            share = s_dev_clac

            move(srcDir, dstDir, share)
            
            # Pour train_clac

            srcDir = os.path.join("./all_clac")
            dstDir = os.path.join("./train/clac")
            share = (s_train_clac - 1) # le - 1 là c'est au cas où all_clac soit impair (du coup il n'y a pas assez d'exemple)

            move(srcDir, dstDir, share)


            # Pour dev_pas_clac

            srcDir = os.path.join("./all_pas_clac")
            dstDir = os.path.join("./dev/pas_clac")
            share = s_dev_pas_clac

            move(srcDir, dstDir, share)
            
            # Pour train_pas_clac

            srcDir = os.path.join("./all_pas_clac")
            dstDir = os.path.join("./train/pas_clac")
            share = (s_train_pas_clac - 1) # le - 1 là c'est au cas où all_clac soit impair (du coup il n'y a pas assez d'exemple)

            move(srcDir, dstDir, share)
            
            
    else:
        break